<a href="https://colab.research.google.com/github/balsrini/Assignment14/blob/master/Copy_of_Copy_of_ExpAp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Took Original Code - 93.1 in 24 epochs 

In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [0]:
tf.enable_eager_execution()


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.55 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from autoaugment import CIFAR10Policy
data = ImageFolder(rootdir, transform=transforms.Compose(
                        [transforms.RandomCrop(32, padding=4, fill=128), # fill parameter needs torchvision installed from source
                         transforms.RandomHorizontalFlip(), CIFAR10Policy(), 
			 transforms.ToTensor(), 
                         Cutout(n_holes=1, length=16), # (https://github.com/uoguelph-mlrg/Cutout/blob/master/util/cutout.py)
                         transforms.Normalize(...)]))
loader = DataLoader(data, ...)

In [0]:
def reinit(normalize = True):
    import torch
    import torchvision
    import torchvision.transforms as transforms
    import numpy as np
    from autoaugment import CIFAR10Policy

    transform = transforms.Compose([            
          #  transforms.Resize(255),
         #   transforms.CenterCrop(224),
            CIFAR10Policy(), 
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            ])

    trainset = torchvision.datasets.CIFAR10(root='/content/gdrive/My Drive/EVA_A14/train', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=5000, shuffle=True, num_workers=0)
    data = iter(trainloader)
    
    train_features = np.zeros((0,32,32,3))
    train_labels = np.zeros((0))
    for i,data in enumerate(trainloader, 0):
        inputs, labels = data         
        #print("Training Data",i," ", train_features.shape)          
        train_features = np.append(train_features,np.transpose(inputs,(0, 2, 3, 1)),axis = 0)        
        train_labels = np.append(train_labels,labels, axis = 0)        
    num_classes = len(np.unique(train_labels))
    #train_labels = np_utils.to_categorical(train_labels, num_classes)
    print(train_labels)
    
    testset = torchvision.datasets.CIFAR10(root='/content/gdrive/My Drive/EVA_A14/test', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=500, shuffle=False, num_workers=0)  
    test_features = np.zeros((0,32,32,3))
    test_labels = np.zeros((0))

    for i,data in enumerate(testloader, 0):
        inputs, labels = data  
        print("test Data",i)          
        test_features = np.append(test_features,np.transpose(inputs,(0, 2, 3, 1)), axis = 0)
        test_labels = np.append(test_labels,labels, axis = 0)     
   
    #test_labels = np_utils.to_categorical(test_labels, num_classes)
    print(test_labels)
    return (num_classes,train_features,train_labels,test_features,test_labels)

In [0]:
#For use in add/concatenate
def Identity(x,f):    
    return add([x,f])

#For use in projection shortcuts
def Projection(x,output_channel,f):    
    return Identity(f,Conv2D(output_channel, (1,1), strides=2, use_bias=False)(x))    

def ConvBN(input,num_channel,kernel_size=(3,3),pad='same',stride=1):
     conv2d = Conv2D(num_channel, kernel_size, strides=stride, padding=pad, use_bias=False)(input)
     bnOutput = BatchNormalization()(conv2d)
     return Activation('relu')(bnOutput)

def ResBlk(input,num_channel,kernel_size=(3,3),pad='same',res = False) :
    conv = ConvBN(input,num_channel)
    h = MaxPooling2D(pool_size=(2, 2))(conv)   
    if res == True:
        res1 = ConvBN(h,num_channel)
        res2 = ConvBN(res1,num_channel)
        h = Identity(h,res2)
    return h

def DavidNet1(img_height,img_width,channel=3) :    
    #Define input
    num_channel = 64
    input_layer = Input(shape=(img_height, img_width, channel))
    init_blk = ConvBN(input_layer,num_channel)
    blk1 = ResBlk(init_blk,num_channel*2,res= True)
    blk2 = ResBlk(blk1,num_channel*4)
    blk3 = ResBlk(blk2,num_channel*8,res=True)
    h = MaxPooling2D(pool_size=(2, 2))(blk3)   
    h =  Flatten()(h)   
    h = Dense(10)(h) 
    output = Activation('softmax')(h)
    model = Model(inputs=[input_layer], outputs=[output])
    model.summary()
    return model

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.keras.layers.Dropout(0.1)(tf.nn.relu(self.bn(self.conv(inputs))))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
from keras.utils import np_utils


(num_classes,x_train,y_train,x_test,y_test) = reinit()

Files already downloaded and verified
Training Data 0   (0, 32, 32, 3)
Training Data 1   (500, 32, 32, 3)
Training Data 2   (1000, 32, 32, 3)
Training Data 3   (1500, 32, 32, 3)
Training Data 4   (2000, 32, 32, 3)
Training Data 5   (2500, 32, 32, 3)
Training Data 6   (3000, 32, 32, 3)
Training Data 7   (3500, 32, 32, 3)
Training Data 8   (4000, 32, 32, 3)
Training Data 9   (4500, 32, 32, 3)
Training Data 10   (5000, 32, 32, 3)
Training Data 11   (5500, 32, 32, 3)
Training Data 12   (6000, 32, 32, 3)
Training Data 13   (6500, 32, 32, 3)
Training Data 14   (7000, 32, 32, 3)
Training Data 15   (7500, 32, 32, 3)
Training Data 16   (8000, 32, 32, 3)
Training Data 17   (8500, 32, 32, 3)
Training Data 18   (9000, 32, 32, 3)
Training Data 19   (9500, 32, 32, 3)
Training Data 20   (10000, 32, 32, 3)
Training Data 21   (10500, 32, 32, 3)
Training Data 22   (11000, 32, 32, 3)
Training Data 23   (11500, 32, 32, 3)
Training Data 24   (12000, 32, 32, 3)
Training Data 25   (12500, 32, 32, 3)
Training

In [0]:
print(y_train.shape)

(50000,)


In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

In [0]:
model = DavidNet()
#model = DavidNet1(32,32,3)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)*3, EPOCHS*2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]
#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5,(((EPOCHS+1)//5)*3)+1, EPOCHS*2], [0, LEARNING_RATE,LEARNING_RATE*0.42, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
model.summary()

Model: "david_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn (ConvBN)             multiple                  1984      
_________________________________________________________________
res_blk (ResBlk)             multiple                  370176    
_________________________________________________________________
res_blk_1 (ResBlk)           multiple                  295936    
_________________________________________________________________
res_blk_2 (ResBlk)           multiple                  5904384   
_________________________________________________________________
global_max_pooling2d (Global multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5120      
Total params: 6,577,600
Trainable params: 6,573,120
Non-trainable params: 4,480
___________________________________________

#LR 0.55 Dropout 0.1 in each conv block

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      #x = tf.cast(x, tf.float32)
      #y = tf.cast(y, tf.int64)
      print(y.shape)
      print(y)
      #print(x.dtype)
      #print(y.dtype)
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

(512,)
tf.Tensor(
[7 5 9 7 0 0 0 7 9 4 2 4 4 1 4 2 7 4 3 8 8 4 1 6 9 2 1 7 7 6 6 3 1 8 2 1 4
 6 9 2 4 2 9 0 9 8 4 4 8 3 2 6 9 9 7 7 5 7 1 9 2 8 5 1 1 7 5 4 3 5 0 1 5 0
 9 5 6 1 1 0 4 1 1 8 6 4 8 8 3 4 2 3 6 5 8 7 5 3 6 4 6 4 7 8 1 1 6 6 7 5 0
 5 0 0 0 6 9 2 8 3 8 6 0 7 9 5 5 8 0 0 5 6 6 7 1 9 0 6 2 7 7 9 9 8 8 5 3 5
 9 7 7 0 3 6 1 8 5 6 8 7 1 1 1 8 1 6 9 6 9 7 9 6 7 4 5 7 8 4 7 1 1 7 8 9 2
 4 7 0 1 2 1 5 4 3 1 3 2 4 6 7 0 9 3 5 0 6 6 4 7 7 9 4 1 3 2 9 3 0 3 6 4 8
 9 8 2 0 9 5 1 0 9 4 3 2 5 4 6 6 5 4 2 0 7 1 6 2 4 0 6 7 5 9 7 8 6 4 6 3 6
 8 2 6 0 0 8 3 5 6 1 3 4 8 9 6 5 7 5 3 8 2 6 3 0 1 9 8 8 3 2 3 8 3 5 0 9 2
 4 0 1 2 4 0 1 9 5 5 1 2 3 2 9 7 8 5 1 9 4 0 0 7 5 6 9 1 8 6 0 9 8 0 7 7 6
 7 8 4 9 1 4 8 1 5 4 6 3 5 4 6 5 2 4 0 3 3 6 8 1 2 0 3 3 3 2 4 6 4 5 7 0 8
 4 7 1 3 6 8 4 7 3 3 9 8 3 8 2 9 9 5 3 2 7 5 8 7 2 6 3 3 2 1 3 5 6 7 8 0 1
 7 2 6 3 1 8 4 5 7 3 6 3 1 1 2 3 0 5 1 2 8 2 5 8 1 8 3 6 7 1 3 8 9 4 0 1 8
 2 9 5 5 3 3 0 3 8 3 1 3 3 1 6 9 8 9 5 0 2 8 3 3 8 3 5 1 8 1 6 2 0 1 4 2 9
 6 1 1 

KeyboardInterrupt: ignored

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      x = tf.cast(x, tf.float32)
      y = tf.cast(y, tf.int64)
      #print(y.shape)
      #print(y)
      #print(x.dtype)
      #print(y.dtype)
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.int64)
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

epoch: 1 lr: 0.11000000000000001 train loss: 0.7397643170166016 train acc: 0.741 val loss: 0.7359258865356445 val acc: 0.749 time: 85.01218867301941


epoch: 2 lr: 0.22000000000000003 train loss: 0.5799529681396485 train acc: 0.8012 val loss: 0.6028373718261719 val acc: 0.7925 time: 169.04264760017395


epoch: 3 lr: 0.33000000000000007 train loss: 0.4860360601806641 train acc: 0.83228 val loss: 0.5585175567626953 val acc: 0.8121 time: 252.9346568584442


epoch: 4 lr: 0.44000000000000006 train loss: 0.4001967150878906 train acc: 0.8634 val loss: 0.4623646827697754 val acc: 0.8373 time: 336.8356742858887


epoch: 5 lr: 0.55 train loss: 0.29643619171142577 train acc: 0.89824 val loss: 0.48755057830810544 val acc: 0.8344 time: 420.72853350639343


epoch: 6 lr: 0.5210526315789474 train loss: 0.23576777099609375 train acc: 0.91832 val loss: 0.5332126312255859 val acc: 0.822 time: 504.7164924144745


#LR 1.0 Dropout 0.1 in each conv block
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)3, EPOCHS2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.2 train loss: 1.5613798889160155 train acc: 0.42812 val loss: 1.5382182281494141 val acc: 0.4668 time: 93.53130340576172



epoch: 2 lr: 0.4 train loss: 0.9691191705322265 train acc: 0.66344 val loss: 0.8635841522216797 val acc: 0.7114 time: 176.85098958015442



epoch: 3 lr: 0.6000000000000001 train loss: 0.7397711651611328 train acc: 0.7469 val loss: 0.7423320419311523 val acc: 0.7446 time: 259.8358778953552



epoch: 4 lr: 0.8 train loss: 0.5949254104614258 train acc: 0.79802 val loss: 0.6257790771484375 val acc: 0.7866 time: 342.9937560558319



epoch: 5 lr: 1.0 train loss: 0.49497475921630857 train acc: 0.8287 val loss: 0.6279651123046875 val acc: 0.7792 time: 426.0994510650635


...


epoch: 20 lr: 0.2946533333333333 train loss: 0.06306992572784424 train acc: 0.97904 val loss: 0.26412418060302734 val acc: 0.9164 time: 1671.9843227863312



epoch: 21 lr: 0.28413 train loss: 0.05816684497833252 train acc: 0.98018 val loss: 0.26152197952270506 val acc: 0.9195 time: 1755.1395869255066



epoch: 22 lr: 0.27360666666666666 train loss: 0.0502595973777771 train acc: 0.98284 val loss: 0.2545664283752441 val acc: 0.9232 time: 1837.9236357212067



epoch: 23 lr: 0.26308333333333334 train loss: 0.048105348110198974 train acc: 0.9833 val loss: 0.2757120216369629 val acc: 0.9186 time: 1921.0395073890686



epoch: 24 lr: 0.25256 train loss: 0.04525558307647705 train acc: 0.98462 val loss: 0.2764810920715332 val acc: 0.9183 time: 2004.7371654510498


#LR 0.9 Dropout 0.1 in each conv block
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)3, EPOCHS2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.18 train loss: 1.55570796875 train acc: 0.43002 val loss: 1.183426449584961 val acc: 0.5707 time: 93.42889618873596



epoch: 2 lr: 0.36 train loss: 0.9808535302734375 train acc: 0.65786 val loss: 0.9287964294433594 val acc: 0.6848 time: 176.16937565803528



epoch: 3 lr: 0.54 train loss: 0.7318989520263672 train acc: 0.74966 val loss: 1.05178701171875 val acc: 0.6545 time: 259.38248324394226



epoch: 4 lr: 0.72 train loss: 0.6096063549804688 train acc: 0.79378 val loss: 0.8324134338378906 val acc: 0.7651 time: 342.1616106033325



epoch: 5 lr: 0.9 train loss: 0.5043676312255859 train acc: 0.82834 val loss: 0.579943733215332 val acc: 0.7949 time: 425.3703308105469


...


epoch: 20 lr: 0.265188 train loss: 0.06541824131011963 train acc: 0.97774 val loss: 0.27498903198242186 val acc: 0.9151 time: 1669.1716232299805



epoch: 21 lr: 0.25571699999999997 train loss: 0.0582493871307373 train acc: 0.97998 val loss: 0.2578946975708008 val acc: 0.9198 time: 1752.4378304481506



epoch: 22 lr: 0.246246 train loss: 0.0526180450630188 train acc: 0.98204 val loss: 0.25819512405395506 val acc: 0.921 time: 1835.1183762550354



epoch: 23 lr: 0.23677499999999999 train loss: 0.04890917444229126 train acc: 0.98326 val loss: 0.26397943115234374 val acc: 0.9188 time: 1918.06999874115



epoch: 24 lr: 0.227304 train loss: 0.045989927005767824 train acc: 0.98416 val loss: 0.26234320793151855 val acc: 0.9214 time: 2001.3373472690582


#LR 0.8 Dropout 0.1 in each conv block
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)3, EPOCHS2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.16 train loss: 1.5634322631835937 train acc: 0.42722 val loss: 1.199383056640625 val acc: 0.5611 time: 92.88046002388



epoch: 2 lr: 0.32 train loss: 0.9377112322998047 train acc: 0.6707 val loss: 0.7365316101074219 val acc: 0.7443 time: 176.16849851608276



epoch: 3 lr: 0.48 train loss: 0.7325146328735351 train acc: 0.74988 val loss: 0.742135986328125 val acc: 0.7514 time: 258.8800392150879



epoch: 4 lr: 0.64 train loss: 0.6150053411865234 train acc: 0.78916 val loss: 0.7267712265014649 val acc: 0.7568 time: 341.95126271247864



epoch: 5 lr: 0.8 train loss: 0.49828393432617185 train acc: 0.82944 val loss: 0.5392093612670898 val acc: 0.8165 time: 424.864209651947


...


epoch: 20 lr: 0.23572266666666666 train loss: 0.06399063621520996 train acc: 0.97822 val loss: 0.28203581161499025 val acc: 0.9151 time: 1667.1918909549713



epoch: 21 lr: 0.227304 train loss: 0.05935045124053955 train acc: 0.97952 val loss: 0.275843461227417 val acc: 0.9189 time: 1750.138682126999



epoch: 22 lr: 0.21888533333333335 train loss: 0.05626008985519409 train acc: 0.98068 val loss: 0.26000304946899416 val acc: 0.9213 time: 1833.0433237552643



epoch: 23 lr: 0.2104666666666667 train loss: 0.05045378126144409 train acc: 0.98358 val loss: 0.2532841194152832 val acc: 0.9227 time: 1916.2136874198914



epoch: 24 lr: 0.202048 train loss: 0.04531113697052002 train acc: 0.98472 val loss: 0.2520535202026367 val acc: 0.9245 time: 1999.4836750030518


#LR 0.4 Dropout 0.1 in each conv block
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)3, EPOCHS2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.6185018328857421 train acc: 0.41108 val loss: 1.1550832855224609 val acc: 0.5746 time: 96.15686321258545



epoch: 2 lr: 0.16 train loss: 0.8981527392578125 train acc: 0.68028 val loss: 0.7728498352050781 val acc: 0.7248 time: 179.0756652355194



epoch: 3 lr: 0.24 train loss: 0.6879570147705079 train acc: 0.7599 val loss: 0.7344525497436524 val acc: 0.7485 time: 262.02633333206177



epoch: 4 lr: 0.32 train loss: 0.6003438854980468 train acc: 0.79456 val loss: 0.6517102935791016 val acc: 0.7802 time: 345.17329144477844



epoch: 5 lr: 0.4 train loss: 0.520707900390625 train acc: 0.8208 val loss: 0.8137483154296875 val acc: 0.7383 time: 428.17586398124695


...


epoch: 20 lr: 0.11786133333333333 train loss: 0.07713578582763672 train acc: 0.97364 val loss: 0.2483159065246582 val acc: 0.9215 time: 1671.7701361179352



epoch: 21 lr: 0.113652 train loss: 0.0733547240447998 train acc: 0.97542 val loss: 0.2516835350036621 val acc: 0.9195 time: 1754.7506177425385



epoch: 22 lr: 0.10944266666666667 train loss: 0.06669550315856934 train acc: 0.97784 val loss: 0.2506546875 val acc: 0.921 time: 1837.6181321144104



epoch: 23 lr: 0.10523333333333335 train loss: 0.06040500612258911 train acc: 0.9802 val loss: 0.26107915954589844 val acc: 0.9175 time: 1920.4592807292938



epoch: 24 lr: 0.101024 train loss: 0.05814304489135742 train acc: 0.98106 val loss: 0.2691509498596191 val acc: 0.9147 time: 2003.4131081104279


#LR 0.7 Dropout 0.1 in each conv block
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS//4)*3, EPOCHS*2], [0, LEARNING_RATE, LEARNING_RATE*0.3157, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.13999999999999999 train loss: 1.5568533074951172 train acc: 0.43278 val loss: 1.3182153503417968 val acc: 0.5315 time: 97.12889671325684



epoch: 2 lr: 0.27999999999999997 train loss: 0.917653226928711 train acc: 0.67622 val loss: 1.109482241821289 val acc: 0.6333 time: 183.7185320854187



epoch: 3 lr: 0.41999999999999993 train loss: 0.7193161328125 train acc: 0.75378 val loss: 0.7703309524536133 val acc: 0.7327 time: 270.5238034725189



epoch: 4 lr: 0.5599999999999999 train loss: 0.6068171176147461 train acc: 0.79202 val loss: 0.5641710311889648 val acc: 0.8106 time: 357.02232217788696



epoch: 5 lr: 0.7 train loss: 0.5065558016967774 train acc: 0.827 val loss: 0.5444045608520508 val acc: 0.8147 time: 443.55707001686096


...


epoch: 20 lr: 0.2062573333333333 train loss: 0.06910953594207764 train acc: 0.97594 val loss: 0.2591572021484375 val acc: 0.9191 time: 1741.7133581638336



epoch: 21 lr: 0.19889099999999996 train loss: 0.06268630863189698 train acc: 0.97882 val loss: 0.2448526020050049 val acc: 0.9225 time: 1828.484111070633



epoch: 22 lr: 0.19152466666666662 train loss: 0.055968160972595216 train acc: 0.98164 val loss: 0.25428817443847657 val acc: 0.9232 time: 1914.9532725811005



epoch: 23 lr: 0.1841583333333333 train loss: 0.050897444229125975 train acc: 0.98288 val loss: 0.24489095611572265 val acc: 0.9266 time: 2001.5719542503357



epoch: 24 lr: 0.17679199999999998 train loss: 0.04662548933029175 train acc: 0.98474 val loss: 0.2491689582824707 val acc: 0.9242 time: 2088.3283457756042


LR 0.7 Dropout at each conv block of 0.1

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.13999999999999999 train loss: 1.5611218591308593 train acc: 0.42894 val loss: 1.060023944091797 val acc: 0.6223 time: 97.58304452896118



epoch: 2 lr: 0.27999999999999997 train loss: 0.9364475805664062 train acc: 0.66744 val loss: 0.867331997680664 val acc: 0.6962 time: 183.85023021697998



epoch: 3 lr: 0.41999999999999993 train loss: 0.7374094342041015 train acc: 0.74536 val loss: 0.6486792846679688 val acc: 0.776 time: 270.3368101119995



epoch: 4 lr: 0.5599999999999999 train loss: 0.6100165850830078 train acc: 0.7905 val loss: 0.615713362121582 val acc: 0.7841 time: 356.7733099460602



epoch: 5 lr: 0.7 train loss: 0.5288307705688476 train acc: 0.82092 val loss: 0.5815406021118164 val acc: 0.81 time: 443.0052342414856


...


epoch: 20 lr: 0.1473684210526316 train loss: 0.06308997379302979 train acc: 0.97842 val loss: 0.24833800354003907 val acc: 0.9219 time: 1736.0246176719666



epoch: 21 lr: 0.1105263157894737 train loss: 0.054133979625701906 train acc: 0.98196 val loss: 0.24073390350341797 val acc: 0.9255 time: 1822.0630524158478



epoch: 22 lr: 0.0736842105263158 train loss: 0.046170203075408936 train acc: 0.98528 val loss: 0.2372231143951416 val acc: 0.9266 time: 1908.2251863479614



epoch: 23 lr: 0.0368421052631579 train loss: 0.039631990718841555 train acc: 0.9874 val loss: 0.2341359146118164 val acc: 0.9277 time: 1994.564793586731



epoch: 24 lr: 0.0 train loss: 0.037008205585479736 train acc: 0.98886 val loss: 0.23392906951904296 val acc: 0.9281 time: 2080.759441137314


#LR 0.45 Dropout at each conv block of 0.1

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.09 train loss: 1.6093720880126954 train acc: 0.41528 val loss: 1.4454825775146485 val acc: 0.5003 time: 100.63132119178772



epoch: 2 lr: 0.18 train loss: 0.9076580300903321 train acc: 0.67364 val loss: 0.9112862823486328 val acc: 0.6843 time: 187.41978454589844



epoch: 3 lr: 0.27 train loss: 0.6964826358032227 train acc: 0.75792 val loss: 0.7242562271118164 val acc: 0.7509 time: 274.3575932979584



epoch: 4 lr: 0.36 train loss: 0.5908196411132812 train acc: 0.7966 val loss: 0.7047820220947265 val acc: 0.7665 time: 360.91990637779236



epoch: 5 lr: 0.45 train loss: 0.5263953076171874 train acc: 0.81786 val loss: 0.5628102294921875 val acc: 0.8068 time: 447.51778769493103


...


epoch: 20 lr: 0.09473684210526317 train loss: 0.07009051982879638 train acc: 0.97646 val loss: 0.24505924377441407 val acc: 0.922 time: 1745.5478811264038



epoch: 21 lr: 0.07105263157894737 train loss: 0.060390997619628906 train acc: 0.98 val loss: 0.2468612060546875 val acc: 0.9249 time: 1832.116574048996



epoch: 22 lr: 0.04736842105263156 train loss: 0.05479046653747559 train acc: 0.9823 val loss: 0.24013196334838868 val acc: 0.9235 time: 1918.5657539367676



epoch: 23 lr: 0.023684210526315808 train loss: 0.04834396759033203 train acc: 0.98514 val loss: 0.23334043960571288 val acc: 0.9256 time: 2005.4083189964294



epoch: 24 lr: 0.0 train loss: 0.042459528408050534 train acc: 0.9875 val loss: 0.23038554763793945 val acc: 0.9277 time: 2092.0251698493958


#LR 0.45 Dropout 0.1 in Resblk 

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.09 train loss: 1.5601539294433593 train acc: 0.4359 val loss: 1.3517421142578125 val acc: 0.4942 time: 86.23734164237976



epoch: 2 lr: 0.18 train loss: 0.8482016885375977 train acc: 0.70082 val loss: 1.1699501953125 val acc: 0.6111 time: 161.45427417755127



epoch: 3 lr: 0.27 train loss: 0.6611993646240234 train acc: 0.76902 val loss: 0.6394796142578125 val acc: 0.7743 time: 236.5605034828186



epoch: 4 lr: 0.36 train loss: 0.5735069894409179 train acc: 0.80372 val loss: 0.7115154220581055 val acc: 0.759 time: 311.54889941215515



epoch: 5 lr: 0.45 train loss: 0.49239184814453124 train acc: 0.83188 val loss: 0.5262205169677734 val acc: 0.8138 time: 386.8485064506531


...


epoch: 20 lr: 0.09473684210526317 train loss: 0.045081437664031984 train acc: 0.98558 val loss: 0.23876489562988282 val acc: 0.9241 time: 1512.9077262878418



epoch: 21 lr: 0.07105263157894737 train loss: 0.039767708778381344 train acc: 0.98802 val loss: 0.2412194496154785 val acc: 0.9235 time: 1587.953899383545



epoch: 22 lr: 0.04736842105263156 train loss: 0.033151666431427 train acc: 0.99034 val loss: 0.24009597930908202 val acc: 0.9245 time: 1663.0420577526093



epoch: 23 lr: 0.023684210526315808 train loss: 0.028036559143066405 train acc: 0.9923 val loss: 0.2337954990386963 val acc: 0.9261 time: 1738.1828064918518



epoch: 24 lr: 0.0 train loss: 0.02451079306602478 train acc: 0.99404 val loss: 0.23388259048461915 val acc: 0.9279 time: 1813.3297502994537


#Lr 0.45 Dropout 0.2
LR -> lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5,(((EPOCHS+1)//5)*3)+1, EPOCHS*2], [0, LEARNING_RATE,LEARNING_RATE*0.42, 0])[0]

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.09 train loss: 1.57620396484375 train acc: 0.43164 val loss: 1.1888732727050781 val acc: 0.5986 time: 86.31883001327515



epoch: 2 lr: 0.18 train loss: 0.9061096075439453 train acc: 0.6752 val loss: 0.8321874237060547 val acc: 0.7181 time: 161.5271134376526



epoch: 3 lr: 0.27 train loss: 0.6999220733642578 train acc: 0.75868 val loss: 0.8144083221435547 val acc: 0.7311 time: 236.47754549980164



epoch: 4 lr: 0.36 train loss: 0.5866278570556641 train acc: 0.79764 val loss: 0.6197058044433593 val acc: 0.7787 time: 311.67305421829224



epoch: 5 lr: 0.45 train loss: 0.5142377703857421 train acc: 0.82534 val loss: 0.8203940368652344 val acc: 0.7302 time: 386.77535128593445


...


epoch: 20 lr: 0.165375 train loss: 0.06943273094177246 train acc: 0.97668 val loss: 0.31403526306152346 val acc: 0.9013 time: 1512.8268704414368



epoch: 21 lr: 0.15946875 train loss: 0.06267417793273926 train acc: 0.97918 val loss: 0.33155574951171873 val acc: 0.9 time: 1587.8615791797638



epoch: 22 lr: 0.1535625 train loss: 0.05580087820053101 train acc: 0.98128 val loss: 0.2915162353515625 val acc: 0.9068 time: 1662.9106621742249



epoch: 23 lr: 0.14765625 train loss: 0.05199963636398315 train acc: 0.98278 val loss: 0.3032761512756348 val acc: 0.9079 time: 1738.1497201919556



epoch: 24 lr: 0.14175 train loss: 0.04990604793548584 train acc: 0.9837 val loss: 0.3211036651611328 val acc: 0.9038 time: 1813.0947365760803


#LR = 0.45 Dropout 0.2

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.09 train loss: 1.579555171508789 train acc: 0.42944 val loss: 1.204464599609375 val acc: 0.5627 time: 85.66133904457092



epoch: 2 lr: 0.18 train loss: 0.9005925115966796 train acc: 0.6798 val loss: 0.7939939392089844 val acc: 0.745 time: 160.77427315711975



epoch: 3 lr: 0.27 train loss: 0.7226691641235351 train acc: 0.75062 val loss: 0.831830973815918 val acc: 0.7132 time: 235.89830613136292



epoch: 4 lr: 0.36 train loss: 0.5926994476318359 train acc: 0.79626 val loss: 0.6883588348388672 val acc: 0.763 time: 311.0882532596588



epoch: 5 lr: 0.45 train loss: 0.5297165399169922 train acc: 0.81936 val loss: 0.7456437606811523 val acc: 0.7513 time: 386.5659351348877


...


epoch: 20 lr: 0.09473684210526317 train loss: 0.05688612693786621 train acc: 0.98146 val loss: 0.27007233581542966 val acc: 0.9135 time: 1511.993358373642



epoch: 21 lr: 0.07105263157894737 train loss: 0.04882907211303711 train acc: 0.98452 val loss: 0.31090074310302734 val acc: 0.9049 time: 1586.909532546997



epoch: 22 lr: 0.04736842105263156 train loss: 0.04177752595901489 train acc: 0.9871 val loss: 0.2732728214263916 val acc: 0.9172 time: 1661.7970368862152



epoch: 23 lr: 0.023684210526315808 train loss: 0.03710679979324341 train acc: 0.98938 val loss: 0.2721889686584473 val acc: 0.9173 time: 1736.7838723659515



epoch: 24 lr: 0.0 train loss: 0.03282971839904785 train acc: 0.9907 val loss: 0.2730170135498047 val acc: 0.9164 time: 1811.842273235321


# LR 0.6 , Dropout 0.2 at start of each Residual block

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.12 train loss: 1.5503961950683594 train acc: 0.4349 val loss: 1.0944946655273438 val acc: 0.6209 time: 88.52296185493469



epoch: 2 lr: 0.24 train loss: 0.8953441680908203 train acc: 0.68426 val loss: 1.016305352783203 val acc: 0.6464 time: 163.30086183547974



epoch: 3 lr: 0.36 train loss: 0.7242632327270507 train acc: 0.75024 val loss: 0.7242274566650391 val acc: 0.7516 time: 238.2757568359375



epoch: 4 lr: 0.48 train loss: 0.5960854135131836 train acc: 0.7957 val loss: 0.765188801574707 val acc: 0.7395 time: 313.02039885520935



epoch: 5 lr: 0.6 train loss: 0.5060210726928711 train acc: 0.82682 val loss: 0.7721700500488281 val acc: 0.7543 time: 387.7818286418915


...


epoch: 20 lr: 0.12631578947368416 train loss: 0.053168347969055174 train acc: 0.98296 val loss: 0.3214733779907227 val acc: 0.9064 time: 1508.6578323841095



epoch: 21 lr: 0.09473684210526312 train loss: 0.04388959846496582 train acc: 0.98634 val loss: 0.26388270645141604 val acc: 0.9176 time: 1583.3777306079865



epoch: 22 lr: 0.06315789473684208 train loss: 0.03743106389045715 train acc: 0.98826 val loss: 0.2854610610961914 val acc: 0.9124 time: 1658.0474393367767



epoch: 23 lr: 0.03157894736842104 train loss: 0.03223006183624268 train acc: 0.99052 val loss: 0.2712393993377685 val acc: 0.9153 time: 1732.8346631526947



epoch: 24 lr: 0.0 train loss: 0.029504635992050172 train acc: 0.9918 val loss: 0.27539930076599123 val acc: 0.9148 time: 1807.6176698207855


#LR 0.6

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.12 train loss: 1.5179136224365235 train acc: 0.45178 val loss: 1.2507713775634766 val acc: 0.5737 time: 81.16321992874146



epoch: 2 lr: 0.24 train loss: 0.8429057946777344 train acc: 0.70192 val loss: 1.1591276977539062 val acc: 0.6359 time: 148.4824755191803



epoch: 3 lr: 0.36 train loss: 0.6575163732910156 train acc: 0.77206 val loss: 3.0269150817871093 val acc: 0.428 time: 215.7170226573944



epoch: 4 lr: 0.48 train loss: 0.5518017004394531 train acc: 0.81054 val loss: 0.7233233871459961 val acc: 0.7712 time: 282.8879165649414



epoch: 5 lr: 0.6 train loss: 0.47467506988525393 train acc: 0.83896 val loss: 0.6750407409667969 val acc: 0.8029 time: 350.3469090461731


...


epoch: 20 lr: 0.12631578947368416 train loss: 0.032077298221588134 train acc: 0.99032 val loss: 0.2886389266967773 val acc: 0.9235 time: 1359.1638293266296



epoch: 21 lr: 0.09473684210526312 train loss: 0.02677502000808716 train acc: 0.99266 val loss: 0.28166539764404297 val acc: 0.9232 time: 1426.5120544433594



epoch: 22 lr: 0.06315789473684208 train loss: 0.022243740577697752 train acc: 0.99402 val loss: 0.2672193130493164 val acc: 0.9283 time: 1493.766727924347



epoch: 23 lr: 0.03157894736842104 train loss: 0.01873995548725128 train acc: 0.99546 val loss: 0.26518502655029297 val acc: 0.9305 time: 1560.9955019950867



epoch: 24 lr: 0.0 train loss: 0.016296080560684205 train acc: 0.9964 val loss: 0.2620594963073731 val acc: 0.9299 time: 1628.401585817337


#LR 0.5

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

epoch: 1 lr: 0.1 train loss: 1.5488606164550782 train acc: 0.44056 val loss: 1.2287797882080078 val acc: 0.5899 time: 76.66130995750427


epoch: 2 lr: 0.2 train loss: 0.8320591763305664 train acc: 0.70616 val loss: 0.9464990478515625 val acc: 0.6777 time: 148.33798718452454


epoch: 3 lr: 0.30000000000000004 train loss: 0.6549269534301758 train acc: 0.77414 val loss: 0.8102503570556641 val acc: 0.7446 time: 219.81618690490723


epoch: 4 lr: 0.4 train loss: 0.5532061856079101 train acc: 0.81054 val loss: 0.6653388656616211 val acc: 0.7813 time: 291.3300144672394


epoch: 5 lr: 0.5 train loss: 0.47010221099853516 train acc: 0.83866 val loss: 1.2076616943359375 val acc: 0.6555 time: 363.0840427875519


...

epoch: 20 lr: 0.10526315789473684 train loss: 0.03387897081375122 train acc: 0.98994 val loss: 0.25265521697998045 val acc: 0.9281 time: 1436.3593254089355


epoch: 21 lr: 0.07894736842105265 train loss: 0.02856337787628174 train acc: 0.99194 val loss: 0.25404673538208006 val acc: 0.9292 time: 1507.991781949997


epoch: 22 lr: 0.052631578947368474 train loss: 0.0228178395652771 train acc: 0.99386 val loss: 0.2549386169433594 val acc: 0.9297 time: 1579.7045834064484


epoch: 23 lr: 0.026315789473684237 train loss: 0.01981497223854065 train acc: 0.99508 val loss: 0.2525083824157715 val acc: 0.9284 time: 1651.2591648101807


epoch: 24 lr: 0.0 train loss: 0.01752871784210205 train acc: 0.99616 val loss: 0.24939640350341796 val acc: 0.9311 time: 1722.7894914150238


In [0]:
bbb

#LR = 0.4

In [0]:
LEARNING_RATE = 0.4

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 2.340544509277344 train acc: 0.09968 val loss: 2.341598846435547 val acc: 0.1001 time: 67.23514890670776



epoch: 2 lr: 0.16 train loss: 2.340467803955078 train acc: 0.0996 val loss: 2.3417118591308594 val acc: 0.1 time: 134.29423570632935



epoch: 3 lr: 0.24 train loss: 2.3401588061523437 train acc: 0.10014 val loss: 2.3417595703125 val acc: 0.1001 time: 201.19970607757568



KeyboardInterrupt

